# Goal
This will train multiple supervised models (i.e., numerical prediction models) used to predict monthly inventory by state and determine which model fits best.

# 0) Setup

In [ ]:
# Import dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras
import clf
from sqlalchemy import create_engine
import matplotlib as plt

# 1) Connect to / Load in SQL Table

In [ ]:

raw_df = pd.read_csv("Resources/.csv")

# 2) Preprocess Data

In [ ]:
X = df.drop('Target', axis=1)
X

In [ ]:
pd.get_dummies(X['season'])

In [ ]:
X_dummies = pd.get_dummies(X)
print(X_dummies.columns)
X_dummies

In [ ]:
y_label = LabelEncoder().fit_transform(df['Target'])
y_label

# 3) Split into Train/Test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Scaling the Data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4) Apply Different Models and Hyperparameterize, and Compare Results
Models to train/test:
- Random forest
- Logistic regression
- Neural networks

Consider the following questions:
- What pros/cons are important to consider for each model?
- Which score(s) make sense for our scenario? Is accuracy the most important score for our model? In other words, should precision, etc. take precedence over other scores to determine the best model?

False Positive: The model predicts a "hot" market (i.e., an increase in inventory), but in reality, it was not hot. This would negatively impact the buyer in facing stiffer competition due to lower inventory, thus potentially paying more for a home than expected or not being able to purchase a home at all.

False Negative: The model predicts a "not hot" market (i.e., a decrease in inventory), but in reality, it was hot. This may negatively impact the buyer in missing out altogether on a housing market with higher inventory.

## Random Forest Model

In [ ]:
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [ ]:
features = clf.feature_importances_
print(features)
plt.bar(x = range(len(features)), height=features)
plt.show()

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

MAKE PREDICTIONS?

In [ ]:
# Generate a new data point (the red circle)
import numpy as np
import matplotlib.pyplot as plt
new_data = np.array([[-2, 6]])
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.scatter(new_data[0, 0], new_data[0, 1], c="r", marker="o", s=100)

In [ ]:
# Predict the class (purple or yellow) of the new data point
predictions = classifier.predict(new_data)
print("Classes are either 0 (purple) or 1 (yellow)")
print(f"The new point was classified as: {predictions}")

In [ ]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

## Neural Networks Model

In [ ]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

In [ ]:
nn_model = tf.keras.models.Sequential()

nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=2))

nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn_model.summary()

In [ ]:
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# 5) Determine Best Model to Utilize
Contextualize why.

# 6) Deploy Model